In [156]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [157]:
temp_dic = {}

for i in range(1, 11):
    if i != 10:
        df = pd.read_table(f'온도_2012-0{i}.txt', encoding='cp949')
        temp_dic[i] = df
    else:
        df = pd.read_table(f'온도_2012-{i}.txt', encoding='cp949')
        temp_dic[i] = df
        
df = pd.read_table('온도_2011-12.txt', encoding='cp949')
temp_dic[0] = df

In [158]:
power_dic = {}

for i in range(1, 11):
    if i != 10:
        df = pd.read_table(f'전력_2012-0{i}.txt', encoding='cp949')
        power_dic[i] = df
    else:
        df = pd.read_table(f'전력_2012-{i}.txt', encoding='cp949')
        power_dic[i] = df

df = pd.read_table('전력_2011-12.txt', encoding='cp949')
power_dic[0] = df

In [159]:
temp_df = pd.concat(temp_dic).reset_index(drop=True)
power_df = pd.concat(power_dic).reset_index(drop=True)

In [160]:
remove_power_list = [2, 216, 262, 301, 302]
remove_temp_list = [152, 267]

remove_power_day = '2012-02-29'

In [161]:
power_df = power_df[power_df['ST_DATE'] != remove_power_day]

In [162]:
power_df = power_df[(power_df['H_ID'] != 2) & 
                    (power_df['H_ID'] != 216) & 
                    (power_df['H_ID'] != 262) & 
                    (power_df['H_ID'] != 301) & 
                    (power_df['H_ID'] != 302)]

In [163]:
temp_df = temp_df[(temp_df['H_ID'] != 152) &
                  (temp_df['H_ID'] != 267)]

In [164]:
power_time_column = []
for i in range(151):
    if i >= 7:
        col = 'power_' + power_df.columns[i]
    else:
        col = power_df.columns[i]
    
    power_time_column.append(col)

    
temp_time_column = []
for i in range(149):
    if i >= 5:
        col = 'temp_' + temp_df.columns[i]
    else:
        col = temp_df.columns[i]
    
    temp_time_column.append(col)

In [165]:
power_df = pd.DataFrame(power_df.values, columns=power_time_column)
temp_df = pd.DataFrame(temp_df.values, columns=temp_time_column)

In [166]:
power_time_col = power_df.columns[7:].tolist()
power_col = power_df.columns[:7].tolist()

temp_time_col = temp_df.columns[5:].tolist()
temp_col = temp_df.columns[:5].tolist()

In [167]:
power_data = power_df.melt(id_vars=power_col, value_vars=power_time_col)

In [168]:
power_data['hour'] = power_data['variable'].apply(lambda x: x.split('_')[1].split('H')[1])
power_data['minute'] = power_data['variable'].apply(lambda x: str(int(x.split('_')[2]) - 1) + '0')
power_data['date_time'] = power_data['ST_DATE'] + ' ' + power_data['hour'] + ':' + power_data['minute']
power_data['date_time'] = pd.to_datetime(power_data['date_time'])

In [169]:
power_data = power_data.drop(['ST_DATE', 'variable', 'hour', 'minute', 'PointName'], axis=1)
power_data = power_data.rename(columns={'value':'power'})

In [170]:
temp_data = temp_df.melt(id_vars=temp_col, value_vars=temp_time_col)

In [171]:
temp_data['hour'] = temp_data['variable'].apply(lambda x: x.split('_')[1].split('H')[1])
temp_data['minute'] = temp_data['variable'].apply(lambda x: str(int(x.split('_')[2]) - 1) + '0')
temp_data['date_time'] = temp_data['ST_DATE'] + ' ' + temp_data['hour'] + ':' + temp_data['minute']
temp_data['date_time'] = pd.to_datetime(temp_data['date_time'])

In [172]:
temp_data = temp_data.drop(['ST_DATE', 'variable', 'hour', 'minute'], axis=1)
temp_data = temp_data.rename(columns={'value':'temp'})

In [173]:
merge_df = power_data.merge(temp_data, how='left', on=['H_ID', 'date_time', 'APT_TF', 'M2', 'PersonCount']).dropna()
merge_df['APT'] = merge_df['APT_TF'].replace({'아파트':'1', '비아파트':'0'})
merge_df['size'] = merge_df['M2'].replace({'1평~19평': '19', '20평~29평': '29', '30평~39평이상': '39'})
merge_df['person'] = merge_df['PersonCount'].replace({'2인이하': '2', '3인': '3', '4인이상': '4'})

In [174]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

encoder.fit(merge_df['HA'])
merge_df['ha'] = encoder.transform(merge_df['HA'])
print(encoder.classes_)

['TV' '가스렌지' '가습기' '감시카메라' '게임기' '공기청정기' '공유기' '김치냉장고' '냉온수기' '냉장고'
 '냉풍기(큰방)' '노트북' '다리미' '데스크탑' '드라이기' '런닝머신' '모니터' '미싱기' '믹서기' '보일러' '비데기'
 '빔프로젝트' '선풍기' '세탁기' '셋톱박스' '수족관' '스탠드' '약탕기' '어항' '에어컨' '오디오' '오븐'
 '전기맛사지기' '전기밥솥' '전기장판' '전기포트' '전기히터' '전력총합' '전자렌지' '정수기' '제본기' '제습기'
 '청소기' '충전기' '커피머신' '커피포트' '토스트기' '프린터' '화장대']


In [175]:
merge_df = merge_df[['date_time', 'H_ID', 'APT', 'size', 'person', 'ha', 'power', 'temp']].sort_values(['date_time', 'H_ID']).reset_index(drop=True)

In [176]:
encoder.classes_[29]

'에어컨'

In [177]:
encoder.classes_[33]

'전기밥솥'

In [178]:
encoder.classes_[23]

'세탁기'

In [179]:
clean_df = merge_df[merge_df['ha'] == 23].reset_index(drop=True)
air_df = merge_df[merge_df['ha'] == 29].reset_index(drop=True)
rice_df = merge_df[merge_df['ha'] == 33].reset_index(drop=True)

In [180]:
print(clean_df.shape)
print(air_df.shape)
print(rice_df.shape)

(1931490, 8)
(293664, 8)
(2226061, 8)


In [54]:
new_air_df = air_df.merge(clean_df, on=['date_time', 'H_ID', 'APT', 'size', 'person', 'temp'], how='left').dropna()

In [57]:
final_df = new_air_df.merge(rice_df, on=['date_time', 'H_ID', 'APT', 'size', 'person', 'temp'], how='left').dropna().reset_index(drop=True)

In [59]:
def holiday(x):
    if x.weekday() == 5 or x.weekday() == 6:
        return 1
    else: 
        return 0

In [60]:
final_df['hoilday'] = final_df['date_time'].apply(lambda x: holiday(x))
final_df['month'] = final_df['date_time'].apply(lambda x: x.month)
final_df['weekday'] = final_df['date_time'].apply(lambda x: x.weekday())
final_df['hour'] = final_df['date_time'].apply(lambda x: x.hour)
final_df['minute'] = final_df['date_time'].apply(lambda x: x.minute // 10)
# df['time'] = df['date_time'].apply(lambda x: x.time())

In [67]:
final_df = final_df.rename(columns={'ha_x':'air', 'ha_y':'washer', 'ha':'rice', 
                                    'power_x':'power_air', 'power_y':'power_washer', 'power':'power_rice'})

In [69]:
final_df.columns

Index(['date_time', 'H_ID', 'APT', 'size', 'person', 'air', 'power_air',
       'temp', 'washer', 'power_washer', 'rice', 'power_rice', 'hoilday',
       'month', 'weekday', 'hour', 'minute'],
      dtype='object')

In [71]:
final_df = final_df[['date_time', 'H_ID', 'APT', 'size', 'person', 
                     'hoilday', 'month', 'weekday', 'hour', 'minute', 
                     'washer', 'power_washer', 'air', 'power_air', 'rice', 'power_rice', 'temp']]

In [78]:
final_df.loc[(final_df['power_washer'] != 0) &
             (final_df['power_rice'] == 0) &
             (final_df['power_air'] == 0), 'washer'] = 1

final_df.loc[final_df['washer'] == 23, 'washer'] = 0

final_df.loc[(final_df['power_washer'] == 0) &
             (final_df['power_rice'] != 0) &
             (final_df['power_air'] == 0), 'rice'] = 1

final_df.loc[final_df['rice'] == 33, 'rice'] = 0

final_df.loc[(final_df['power_washer'] == 0) &
             (final_df['power_rice'] == 0) &
             (final_df['power_air'] != 0), 'air'] = 1

final_df.loc[final_df['air'] == 29, 'air'] = 0

In [99]:
final_df.loc[(final_df['power_washer'] == 0) &
             (final_df['power_rice'] != 0) &
             (final_df['power_air'] != 0), 'air_rice'] = 1

final_df['air_rice'] = final_df['air_rice'].fillna(0)

final_df.loc[(final_df['power_washer'] != 0) &
             (final_df['power_rice'] == 0) &
             (final_df['power_air'] != 0), 'air_washer'] = 1

final_df['air_washer'] = final_df['air_washer'].fillna(0)

final_df.loc[(final_df['power_washer'] != 0) &
             (final_df['power_rice'] != 0) &
             (final_df['power_air'] == 0), 'washer_rice'] = 1

final_df['washer_rice'] = final_df['washer_rice'].fillna(0)

final_df.loc[(final_df['power_washer'] != 0) &
             (final_df['power_rice'] != 0) &
             (final_df['power_air'] != 0), 'all'] = 1

final_df['all'] = final_df['all'].fillna(0)

final_df.loc[(final_df['power_washer'] == 0) &
             (final_df['power_rice'] == 0) &
             (final_df['power_air'] == 0), 'all_not'] = 1

final_df['all_not'] = final_df['all_not'].fillna(0)

In [115]:
final_df = final_df[['date_time', 'H_ID', 'APT', 'size', 'person', 'hoilday', 'month',
                     'weekday', 'hour', 'minute', 'power_washer', 'power_air', 'power_rice', 'temp',
                     'washer', 'air', 'rice', 'air_rice', 'air_washer', 'washer_rice', 'all', 'all_not']]

In [151]:
final_df

,date_time,H_ID,APT,size,person,hoilday,month,weekday,hour,minute,...,power_rice,temp,washer,air,rice,air_rice,air_washer,washer_rice,all,all_not
0,2012-06-01 00:00:00,28,0,29,2,0,6,4,0,0,...,0.042,30.600000,0,0,0,0,0,1,0,0
1,2012-06-01 00:00:00,62,1,39,4,0,6,4,0,0,...,0.006,30.600000,0,0,1,0,0,0,0,0
2,2012-06-01 00:00:00,197,1,19,2,0,6,4,0,0,...,0.036,30.600000,0,0,1,0,0,0,0,0
3,2012-06-01 00:10:00,28,0,29,2,0,6,4,0,1,...,0.008,30.600000,0,0,0,0,0,1,0,0
4,2012-06-01 00:10:00,62,1,39,4,0,6,4,0,1,...,0.013,30.600000,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78837,2012-10-14 23:40:00,29,1,19,2,1,10,6,23,4,...,0.004,33.200001,0,0,0,0,0,0,1,0
78841,2012-10-14 23:50:00,29,1,19,2,1,10,6,23,5,...,0.007,33.200001,0,0,0,0,0,0,1,0
78842,2012-10-14 23:50:00,62,1,39,4,1,10,6,23,5,...,0.000,28.100000,0,1,0,0,0,0,0,0
78840,2012-10-14 23:50:00,1,1,29,2,1,10,6,23,5,...,0.000,28.100000,0,0,0,0,0,0,0,1


In [150]:
for i in final_df.columns[14:]:
    final_df[i] = final_df[i].astype('int32')

In [133]:
for i in final_df.columns[10:14]:
    final_df[i] = final_df[i].astype('float32')

In [139]:
for i in final_df.columns[5:10]:
    final_df[i] = final_df[i].astype('int32')

In [152]:
final_df.dtypes

date_time       datetime64[ns]
H_ID                    object
APT                     object
size                    object
person                  object
hoilday                  int32
month                    int32
weekday                  int32
hour                     int32
minute                   int32
power_washer           float32
power_air              float32
power_rice             float32
temp                   float32
washer                   int32
air                      int32
rice                     int32
air_rice                 int32
air_washer               int32
washer_rice              int32
all                      int32
all_not                  int32
dtype: object

In [31]:
# m_df = pd.DataFrame()

# for i in air_df.H_ID.unique():
#     data = air_df[air_df['H_ID'] == i]
#     value = np.array(data['power'].apply(lambda x: 1 if x != 0 else 0))

#     lst_value = list(value)
#     for i in range(3):
#         lst_value.append(0)

#     value = np.array(lst_value)
#     data['use'] = value[3:]
    
#     m_df = pd.concat([m_df, data])

<ipython-input-31-af02ef52b173>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['use'] = value[3:]


In [153]:
final_df = final_df.sort_values('date_time')

In [155]:
final_df.to_csv('최종데이터.csv', index=False)